[Myersurl]: https://desi.lbl.gov/DocDB/cgi-bin/private/ShowDocument?docid=6693 "Myers et al. paper"

# How to run target selection for the DESI Main Survey #
### Author: Adam D. Myers, University of Wyoming ###

This notebook describes how to produce target selection files for DESI, focusing on the Main Survey. For extra detail about different target classes and input and output files, see [Myers et al. (2022)][Myersurl]. 

# Setting up your environment #

If you are part of the DESI collaboration, refer to the notes on the wiki about getting started running the DESI pipelines [at NERSC](https://desi.lbl.gov/trac/wiki/Pipeline/GettingStarted/NERSC) or [on your laptop](https://desi.lbl.gov/trac/wiki/Pipeline/GettingStarted/Laptop).

If you are not part of the DESI collaboration then, at a minimum, you will need to create a Python 3.8 environment using Anaconda (called, e.g., `desi`):

```
conda create --name desi --yes python=3.8 numpy scipy astropy pyyaml requests ipython h5py scikit-learn matplotlib numba sqlalchemy
conda activate desi
```

and additionally install `fitsio`, `healpy` and `photutils` outside of conda:

```
pip install fitsio healpy photutils
```

Wherever you're working, you will need to clone the `desitarget` software and its related dependencies. For example (you can set this `$DESI_PRODUCT_ROOT` to point to any directory that you'd prefer):

```
setenv DESI_PRODUCT_ROOT $HOME/git/
mkdir -p $DESI_PRODUCT_ROOT
cd $DESI_PRODUCT_ROOT
for package in desiutil desitarget desimodel; do
  git clone https://github.com/desihub/$package
done
```

You will then have to point your `$PATH` and `$PYTHONPATH` to the cloned repositories. For example:

```
cd $DESI_PRODUCT_ROOT
for package in desiutil desitarget desimodel; do
  setenv PATH $DESI_PRODUCT_ROOT/$package/bin:$PATH
  setenv PYTHONPATH $DESI_PRODUCT_ROOT/$package/py:$PYTHONPATH
done
```

If you are not working at NERSC, you will also need to download a large number of data files and set environment variables to point to them:

```
setenv DESI_ROOT /global/cfs/cdirs/desi
setenv TARG_DIR $env(DESI_ROOT)/target/catalogs
setenv MTL_DIR  $env(DESI_ROOT)/target/surveyops/mtl
setenv DUST_DIR $env(DESI_ROOT)/external/dust/v0_1
setenv CMX_DIR  $env(DESI_ROOT)/target/cmx_files
setenv MASK_DIR $env(DESI_ROOT)/target/masks
setenv GAIA_DIR $env(DESI_ROOT)/target/gaia_dr2
setenv URAT_DIR $env(DESI_ROOT)/target/urat_dr1
setenv TYCHO_DIR $env(DESI_ROOT)/target/tycho_dr2
setenv TOO_DIR  $env(DESI_ROOT)/target/ToO
setenv SCND_DIR $CSCRATCH/target/secondary
setenv WEB_DIR $CSCRATCH/www
```

Here, I've set the `$DESI_ROOT` environment variable to point to the actual NERSC location of the data files. If you are working elsewhere, you will need to download the directory tree and set it up somewhere on your local network. All of the data files will eventually become public in a future DESI data release, where `$DESI_ROOT` is currently expected to correspond to `https://data.desi.lbl.gov/public/svda/`.

The `$CSCRATCH` directory refers to any scratch space to which you have read/write access. It will be necessary to download (or copy) files from `$DESI_ROOT/target/secondary` to this `$CSCRATCH` space if you want to try to process secondary targets.

You will also need to download a copy of the full [DR9 Legacy Surveys directory tree](https://portal.nersc.gov/cfs/cosmo/data/legacysurvey/dr9/) and point the environment variable `$LS_DIR` to the root `dr9` directory. The directory already exists at NERSC at:

```
setenv LS_DIR /global/project/projectdirs/cosmo/data/legacysurvey/dr9
```

***Note: The remainder of this tutorial will frequently refer to the environment variables that were defined in this section.***

# Basic production of target files at the command line #

First, swap the currently loaded version of `desitarget` for a tagged version (`1.1.1` corresponds to the version of `desitarget` used to process the Main Survey; see [Myers et al. 2022][Myersurl]). If you are working at NERSC and have your environment set up and sourced then you can issue:

```
module swap desitarget/1.1.1
```

Otherwise you can change into the git repository where you cloned `desitarget` and issue:

```
git checkout 1.1.1
```

Note that the _backup_ targets for the Main Survey were run with version `2.2.0` of the `desitarget` code. So, to _also_ process the backup program requires running target files twice.

From this point forward, we will be running production-level code. If you are at NERSC, you may want to familiarize yourself with how to run batch jobs. A straightforward way to run the code is to request interactive nodes. For example, to reserve one interactive Cori Haswell node for 2 hours and 5 minutes at NERSC:

```
salloc -N 1 -C haswell -t 02:05:00 --qos interactive -L SCRATCH,project
```

If you are on your own network or machine, just be aware that it can take significant resources to run the DESI target selection code!

Now, let's produce a file of targets and write it to `$CSCRATCH` (or your equivalent scratch space if you aren't at NERSC):

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --nside 8 --healpixels 444 --numproc 4 --nosec
```

This will read the Legacy Surveys sweep files as inputs, and produce a target file in HEALPixel `444` (in the nested scheme) at `nside` of `8` using `4` processors (`numproc`). The choice of `nside=8` corresponds to about 50 square degrees, which is reasonably efficient while also producing ouput files of a manageable size. The choice of `--numproc 4` is a decent balance between efficiency and memory use on NERSC compute nodes.

The previous command will _not_ produce secondary targets (`--nosec`). If you have set up your environment to include the secondary targeting directory (see **setting up your environment**) and want to also try to process secondary targets, you would instead run:

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --nside 8 --healpixels 444 --numproc 4 --scnddir $SCND_DIR
```

If you wanted to substitute Gaia EDR3 astrometric quantities in place of Gaia DR2 (see Section 4.1.4 of [Myers et al. 2022][Myersurl]) then you would add the `--gaiasub` flag. If you wanted to guarantee that you recover the same `SUBPRIORITY` values as used for the Main Survey (again see [Myers et al. 2022][Myersurl]), then you would add the location of the directories that host frozen `SUBPRIORITY` values. The entire call would then resemble:

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --nside 8 --healpixels 444 --numproc 4 --gaiasub --scnddir $SCND_DIR --dark_subpriorities $TARG_DIR/subpriority/fba-version-4.0.0/subpriorities-dark.fits --bright_subpriorities $TARG_DIR/subpriority/fba-version-4.0.0/subpriorities-bright.fits
```

You should now have produced output target files in `$CSCRATCH/dr9/1.1.1/targets/main/`. This will correspond to the targets in HEALPixel number `444` at `nside` of `8`. If you want to process targets across a wider area of the sky, you will have to run the code multiple times with different HEALPixel numbers — likely by parallelizing the code, as outlined in the next section.

In addition to the `select_targets` command line code, there are `select_cmx_targets` and `select_sv_targets` versions that can be used for commissioning and SV.

# Parallelizing the target selection code #

In the last section, we introduced the `numproc` flag that can be set to thread the target selection code across multiple processors for an individual HEALPixel. As `desitarget` typically splits processing by HEALPixel, it is also straightforward to parallelize across multiple compute nodes on a cluster. 

The `bundlefiles` flag can be used to produce an example script as an aid to parallelizing across HEALPixels. For example, passing `select_targets` the `bundlefiles` option with `1` (assigning one HEALPixel to each node) and a HEALPixel `nside` of `2`:

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH --bundlefiles 1 --nside 2
```

will produce the following output:

```
INFO:cuts.py:3007:select_targets: Running on the main survey
#######################################################
Possible salloc command if you want to run on the Cori interactive queue:

salloc -N 16 -C haswell -t 01:00:00 --qos interactive -L SCRATCH,project

#######################################################
Example shell script for slurm:

#!/bin/bash -l
#SBATCH -q regular
#SBATCH -N 16
#SBATCH -t 01:00:00
#SBATCH -L SCRATCH,project
#SBATCH -C haswell

srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 0 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 1 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 2 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 3 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 4 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 5 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 6 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 7 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 8 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 9 --numproc 32 &
srun -N 1 select_targets /global/project/projectdirs/cosmo/data/legacysurvey/dr9/south/sweep/9.0 $CSCRATCH  --nside 2 --healpixels 10 --numproc 32 &

(etc. down to --healpixels 47)
```

The first few lines of this output include a suggested `salloc` command for requesting interactive nodes (which is typically the easiest way to run `desitarget` code at NERSC). The rest of the output provides an example script to run on the requested nodes. For example, to use this information at NERSC, you would place everything below `#!/bin/bash -l` in a file called, say, `script.sh`. You would then execute `salloc -N 16 -C haswell -t 01:00:00 --qos interactive -L SCRATCH,project` at the command line, wait to be allocated nodes, and then run `script.sh` on your allocated nodes.

The previous section of this document included a typical example of how to run DESI target selection within a single HEALPixel. To generate a script to similarly run target selection but across the full sky, you would repeat the command from the previous section, but add `--bundlefiles 1`:

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --nside 8 --healpixels 444 --numproc 4 --gaiasub --scnddir $SCND_DIR --dark_subpriorities $TARG_DIR/subpriority/fba-version-4.0.0/subpriorities-dark.fits --bright_subpriorities $TARG_DIR/subpriority/fba-version-4.0.0/subpriorities-bright.fits --bundlefiles 1
```

The ```bundlefiles``` option _really_ specifies the number of input sweeps files that will be parallelized across (i.e. for `--bundlefiles 100` the input sweep files would be bundled across HEALPixels so that approximately 100 files touch each pixel). In general, though, `desitarget` codes are typically run with `--bundlefiles 1` to produce one output file of targets per HEALPixel.

Note that the information in this section also applies to the ```select_sv_targets``` command line script that can be used to run targets for SV.

# Useful commands for reading and producing subsets of targets #

In addition to producing subsets of targets in a given HEALPixel, `desitarget` has tools for producing subsets of targets limited by Right Ascension (RA) and Declination (Dec). Additionally, `desitarget` includes wrappers to read subsets of the HEALPixel-partitioned targets produced from a parallelized run of `select_targets`.

### Reading a subset of target files ###

The `desitarget` code has several useful wrappers to read the output of parellelized or "bundled" production runs. To demonstrate this functionality, lets assume that you've downloaded the HEALPixel-split DESI Main Survey dark-time target files (again, see [Myers et al. 2022][Myersurl]) and have placed them at:

```
$TARG_DIR/dr9/1.1.1/targets/main/resolve/dark/
```

The main options include reading targets within; a subset of HEALPixels; a "box" in RA/Dec; or a cap (a "circle" on the sky) specified by RA/Dec/radius in degrees. Let's enter the Python prompt and try each of these options:

In [4]:
# ADM set up the directory from which to read targets.
import os
hpdirname = os.path.join(os.getenv("TARG_DIR"), "dr9", "1.1.1", "targets", "main", "resolve", "dark")

# ADM The various pieces of code for reading in targets.
from desitarget.io import read_targets_in_hp, read_targets_in_box, read_targets_in_cap

# ADM Read targets by HEALPixel.
# ADM The quick=True option ignores any error checks.
# ADM quick=True is always safe to pass for target files that are in the "official" format.
nside, pixlist = 16, [2303, 2557, 3063]
hptargs = read_targets_in_hp(hpdirname, nside, pixlist, quick=True)

# ADM Read targets in an RAmin, RAmax, Decmin, Decmax "rectangle".
radecbox = [120,122,32,34]
boxtargs = read_targets_in_box(hpdirname, radecbox, quick=True)

# ADM Read targets in an RA, Dec, radius (degrees) "circle".
radecrad = [121,33,0.5]
captargs = read_targets_in_cap(hpdirname, radecrad, quick=True)

The default option for extracting targets is to return all columns in a target file, but it's also possible to return just a subset of columns by passing them as a list:

In [5]:
columns = ["RA", "DEC", "DESI_TARGET", "TARGETID"]
hptargs = read_targets_in_hp(hpdirname, nside, pixlist, quick=True, columns=columns)
boxtargs = read_targets_in_box(hpdirname, radecbox, quick=True, columns=columns)
captargs = read_targets_in_cap(hpdirname, radecrad, quick=True, columns=columns)

### Running a subset of target files ###

Reading a subset of targets from previously processed HEALPixel-split files is certainly quicker than reading in all of the targets across the entire DESI footprint. But, it may be even quicker in some situations to *only process that subset* of targets in the first place.

The `desitarget` codebase includes functionality to process target files only in certain regions of the sky, to circumvent the need for a full production run. We have already seen how to run targeting in a subset of HEALPixels. But, there are also options to run in an RA/Dec "box", or a cap (a "circle" on the sky) specified by RA/Dec/radius in degrees. If you're working at NERSC, it's good practice to reserve an interactive node before trying these options:

```
salloc -N 1 -C haswell -t 00:20:00 --qos interactive -L SCRATCH,project
```

Now, here's a basic example of processing targets in an (RAmin, RAmax, Decmin, Decmax) "box":

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH --nosec --radecbox 120,122,32,34
```

and for a cap (a "circle" on the sky) specified by RA/Dec/radius in degrees:

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH --nosec --radecrad 121,33,0.5
```

In both of these cases, the output files will resemble:
```
$CSCRATCH/dr9/1.1.1/targets/main/resolve/bright/targets-bright-hp-X.fits
$CSCRATCH/dr9/1.1.1/targets/main/resolve/dark/targets-dark-hp-X.fits
$CSCRATCH/gaiadr2/1.1.1/targets/main/resolve/backup/targets-backup-hp-X.fits
```

Here, the `hp-X` simply means that the code was not run in a specified set of HEALPixels, i.e., a "special case" was chosen for processing.

Note that all of the above options will also work with the SV version of the code (`select_sv_targets`) in addition to the Main Survey version of the code (`select_targets`).

[Myersurl]: https://desi.lbl.gov/DocDB/cgi-bin/private/ShowDocument?docid=6693 "Myers et al. paper"
[Schlaflyurl]: https://desi.lbl.gov/DocDB/cgi-bin/private/ShowDocument?docid=XXXX "Schlafly et al. paper"

# Production of all desitarget files at the command line #

In addition to the basic files of targets, you may want to process the other `desitarget` products discussed in Section 4 of [Myers et al. (2022)][Myersurl].

## Random catalogs ##

To produce a set of random catalogs, first run the `select_randoms` script using the `bundlebricks` command-line option, as discussed under **Parallelizing the target selection code**:

```
select_randoms $LS_DIR $CSCRATCH --numproc 16 --nside 2 --bundlebricks 15665 --seed 1 --density 50000 -n 20
```

Here, the value of `15665` — which was used to produce random catalogs for [dr9 of the Legacy Surveys](https://www.legacysurvey.org/dr9/files/#random-catalogs-randoms) — efficiently parallelizes across HEALPixels. This value could be altered to try to produce a script that is balanced across your particular set of available nodes or resources. On Cori at NERSC, a set of 20 random catalogs typically takes about 4 hours to run parallelized across about 25 nodes.

As with parallelizing files of targets, the first part of the output from the bundling command provides some suggestions for the resources that will be needed to process the random catalogs. The second part is an example shell script. If you need to know how to generate a smaller random catalog rather than pursuing a full production run, then it should be illustrative to consult one of the commands written to the output bundling script.

The `-n 20` in the "bundling" command indicates that the script should produce 20 distinct random catalogs. The output catalogs will be named `randoms-1-[0-19].fits`, where `1` indicates the specified `seed` and `[0-19]` represents the  specified 20 catalogs. If a higher density of random points is desired (`--density 50000` will produce 50,000 points per sq. deg.), then the "bundling" command can be run with `--seed 2`, `--seed 3`, `--seed 4`, etc. until sufficient catalogs have been generated.

The random catalogs that we have produced, so far, populate the footprint of the DESI Legacy Surveys imaging. To also create random catalogs that contain locations which lie *outside* of the footprint (see Section 4.5.1 of [Myers et al. 2022][Myersurl]), the following set of commands must be run:

```
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-0.fits $CSCRATCH --seed 0
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-1.fits $CSCRATCH --seed 1
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-2.fits $CSCRATCH --seed 2
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-3.fits $CSCRATCH --seed 3
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-4.fits $CSCRATCH --seed 4
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-5.fits $CSCRATCH --seed 5
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-6.fits $CSCRATCH --seed 6
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-7.fits $CSCRATCH --seed 7
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-8.fits $CSCRATCH --seed 8
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-9.fits $CSCRATCH --seed 9
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-10.fits $CSCRATCH --seed 10
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-11.fits $CSCRATCH --seed 11
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-12.fits $CSCRATCH --seed 12
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-13.fits $CSCRATCH --seed 13
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-14.fits $CSCRATCH --seed 14
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-15.fits $CSCRATCH --seed 15
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-16.fits $CSCRATCH --seed 16
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-17.fits $CSCRATCH --seed 17
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-18.fits $CSCRATCH --seed 18
supplement_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-19.fits $CSCRATCH --seed 19
```
This will produce one `randoms-outside` catalog to match each of the "inside" catalogs discussed above. If additional random catalogs have been produced using different seeds, then these commands would need to be repeated but replacing each occurrence of `randoms-1` with, e.g., `randoms-2`, `randoms-3`, `randoms-4`, etc.

The `desitarget` code can also combine the "inside" and "outside" random catalogs to produce `randoms-allsky` catalogs. The following set of commands would be used for our example processing run:

```
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-0
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-1
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-2
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-3
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-4
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-5
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-6
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-7
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-8
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-9
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-10
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-11
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-12
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-13
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-14
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-15
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-16
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-17
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-18
combine_randoms $CSCRATCH/dr9/1.1.1/randoms/resolve/ -s 1-19
```

Again, if catalogs have been processed with `--seed 2`, `--seed 3`, `--seed 4`, etc., in the initial bundling step, then additional `-s` commands would be needed where, e.g., `-s 4-13` means "my initial bundling seed was `4` and this is catalog number `n=13`".

Supplementing and combining the random catalogs is typically fast — taking about 30 minutes per set of 20 on a single Cori node at NERSC.

## pixweight files ##

To produce a HEALPix map of systematics, target densities and areal weights by combining a random catalog with directories of HEALPixel-split dark-time and bright-time targets, issue the following commands:

```
make_imaging_weight_map $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-0.fits $CSCRATCH/dr9/1.1.1/targets/main/resolve/bright/ $CSCRATCH/pixweight-1-bright.fits
make_imaging_weight_map $CSCRATCH/dr9/1.1.1/randoms/resolve/randoms-1-0.fits $CSCRATCH/dr9/1.1.1/targets/main/resolve/dark/ $CSCRATCH/pixweight-1-dark.fits
```

These commands assume that all of the output files from previous sections of this notebook have been stored in the $CSCRATCH directory. 

Note that if you want to produce a random catalog with a higher density of points to more precisely calculate systematics (rather than just using `randoms-1-0.fits`) then the `gather_targets` command line script can be used to concatenate a number of random catalogs, as follows:

```
gather_targets "randoms-1-0.fits;randoms-1-1.fits;randoms-1-2.fits;randoms-1-3.fits;randoms-1-4.fits;randoms-1-5.fits;randoms-1-6.fits;randoms-1-7.fits;randoms-1-8.fits;randoms-1-9.fits;randoms-1-10.fits;randoms-1-11.fits;randoms-1-12.fits;randoms-1-13.fits;randoms-1-14.fits;randoms-1-15.fits;randoms-1-16.fits;randoms-1-17.fits" $CSCRATCH/randoms-1-01234567891011121314151617.fits randoms --norand --col pixweight
```
This command is the origin of the files in the `randomsall` subdirectory (e.g., `$TARG_DIR/dr9/0.49.0/randoms/randomsall/`) discussed in Section 4.5 of [Myers et al. (2022)][Myersurl].

Processing pixweight files is relatively fast. At NERSC, each pixweight file takes about 1 hour to run on one Cori node with a single process per node.

## Quality Assurance (QA) web pages ##
 
The QA web pages discussed in Section 4.5.3 of [Myers et al. (2022)][Myersurl] are produced from the pixweight files and HEALPixel-split directories of dark-time and bright-time targets. To reduce memory requirements, it is convenient to first prepare smaller, interim target files via the following commands:

```  
run_target_qa $CSCRATCH/dr9/1.1.1/targets/main/resolve/bright $CSCRATCH/bright.fits --prepare
run_target_qa $CSCRATCH/dr9/1.1.1/targets/main/resolve/dark $CSCRATCH/dark.fits --prepare
```  

Then, the QA pages can be constructed as follows:

```
run_target_qa $CSCRATCH/bright.fits $WEB_DIR/1.1.1/main/desitargetQA-dr9-bright-1.1.1 --numproc 2 -w $CSCRATCH/pixweight-1-bright.fits
run_target_qa $CSCRATCH/dark.fits $WEB_DIR/1.1.1/main/desitargetQA-dr9-dark-1.1.1 --numproc 2 -w $CSCRATCH/pixweight-1-dark.fits
```

Each of these commands takes about 2.5 hours to complete at NERSC on one Cori node with a single process per node.

## Sky targets ##

Production-level runs of blank sky locations are most easily conducted using information generated by the `bundlebricks` command-line option, as also discussed above for random catalogs and under **parallelizing the target selection code**:

```
select_skies $LS_DIR/south/ $CSCRATCH -s2 $LS_DIR/north/ --bundlebricks 1 --nside 8 --nskiespersqdeg 18000
```

As with other "bundling" commands, this will generate output comprising a script that can be used to parallelize the production of sky files and, as the first part of the output, some useful information outlining how long the script will take to run on NERSC resources. A full production run of sky files at NERSC typically takes about 1-2 hours to run on ~45 Cori nodes with 32 processes per node. The `--nskiespersqdeg 18000` flag specifies that 18000 sky locations should be generated per sq. deg. — this was the value used to process sky locations for the DESI Main Survey.

Similarly to the target files (see also Section 5 of [Myers et al 2022][Myersurl]), it is possible to force the output sky locations to have identical values of `SUBPRIORITY` to those used for the Main Survey production run. This is achieved by adding the `--sky-subpriorities` flag, as follows:

```
select_skies $LS_DIR/south/ $CSCRATCH -s2 $LS_DIR/north/ --bundlebricks 1 --nside 8 --nskiespersqdeg 18000 --sky-subpriorities $TARG_DIR/subpriority/fba-version-4.0.0/subpriorities-sky.fits
```

Because sky locations are generated in Legacy Surveys bricks, it is necessary to recast them in HEALPixels (see, also, Section 4.4.1 of [Myers et al 2022][Myersurl]). This can be achieved using the single command:

```
repartition_skies $CSCRATCH/dr9/1.1.1/skies
```
where `$CSCRATCH/dr9/1.1.1/skies` is the location to which `select_skies` should have written the HEALPixel-split sky files. Repartitioning skies is very fast.

In addition to standard, pixel-level sky locations, a full production run of skies includes supplemental sky locations generated by avoiding bright stars from Gaia (see Section 4.4.2 of [Myers et al 2022][Myersurl]). These supplemental files can be processed via a "bundle" script generated using a similar command to that used for pixel-level sky locations:

```
supplement_skies $CSCRATCH/dr9/1.1.1/skies $CSCRATCH --bundlefiles 1 --nside 8 --nskiespersqdeg 18000 --sky-subpriorities $TARG_DIR/subpriority/fba-version-4.0.0/subpriorities-sky.fits
```

In addition to `supplement_skies` requiring a different input directory to `select_skies` — because the input is the pixel-level sky locations themselves rather than files from the Legacy Surveys — it is important to note that the bundling option for `supplement_skies` is called `--bundlefiles` rather than `--bundlebricks` — because the parallelization is, indeed, conducted across HEALPixels represented in *files* rather than across Legacy Surveys *bricks*. At NERSC, creating supplemental sky locations takes about an hour using 4 Cori production nodes.

## GFA targets ##

Targets for guiding, focusing and alignment (GFA targets) are produced using the `select_gfas` command-line option. The overall process is similar to that for `select_targets`, with a `bundlefiles` option being used to produce a parallelizable script:

```
select_gfas $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --bundlefiles 1 --nside 8 --numproc 8 --gaiadr edr3
```  
Here, the `--gaiadr edr3` option specifies that GFAs should be selected using Gaia EDR3. The default if this flag is omitted is to use Gaia DR2. As for `select_targets`, the output from `select_gfas` when the "bundle" option is specified includes information about how to run GFA targets on NERSC resources and a script for parellizing the production of files. If you have followed the suggested schema in this notebook of using version `1.1.1` of the `desitarget` code to reproduce DESI Main Survey selection for bright-time and dark-time targets, then the GFA files will be written to `$CSCRATCH/dr9/1.1.1/gfas`
At NERSC, a full set of GFA target files takes about an hour to produce when parallelized across 16 Cori nodes.

## Secondary targets ##

If a secondary directory (`$SCND_DIR`) was passed when processing primary targets (see the `select_targets` command in the **basic production of target files at the command line** section), then secondary targets will have been written to `$SCND_DIR` and matches between secondary and primary targets will have been written to `$SCND_DIR/outdata/1.1.1/priminfo-dr9-1.1.1/`.

Monolithic files of "standalone" secondary targets (see Section 4.6.1 of [Myers et al 2022][Myersurl]) can then be (rapidly) produced with the following command:

```
select_secondary $SCND_DIR/outdata/1.1.1/priminfo-dr9-1.1.1/ $CSCRATCH --scnddir $SCND_DIR
```

The resulting output files will be written to `$CSCRATCH/dr9/1.1.1/targets/main/secondary/` with one subdirectory for dark-time targets and one for bright-time targets. As with other target classes, it is possible to force each target to have the same value of `SUBPRIORITY` as was used for the Main Survey by adding flags that point to subpriority files, as follows:

```
select_secondary $SCND_DIR/outdata/1.1.1/priminfo-dr9-1.1.1/ $CSCRATCH --scnddir $SCND_DIR --dark-subpriorities $TARG_DIR/subpriority/fba-version-4.0.0/subpriorities-dark.fits --bright-subpriorities $TARG_DIR/subpriority/fba-version-4.0.0/subpriorities-bright.fits
```

## MTL ledgers ##

As detailed in [Schlafly et al. (2022)][Schlaflyurl], DESI observations are actually scheduled using MTL ("Merged Target List") ledgers rather than the larger target files described above. These ledgers are updated as the DESI survey progresses to reflect the observational state of each target. Assuming that any output target files have been stored in `$CSCRATCH`, then the following commands can be issued to recreate the initial ledgers used for the DESI Main Survey:

```
make_initial_mtl_ledger $CSCRATCH/dr9/1.1.1/targets/main/resolve/dark --dest $CSCRATCH --obscon DARK --timestamp 2021-05-13T08:15:37+00:00 --numproc 8
make_initial_mtl_ledger $CSCRATCH/dr9/1.1.1/targets/main/resolve/bright --dest $CSCRATCH --obscon BRIGHT --timestamp 2021-05-13T08:15:37+00:00 --numproc 8
```

Here, `--timestamp` indicates an initial UTC/ISO time to be written as the `TIMESTAMP` column for each target in the ledger. The specified value of `2021-05-13T08:15:37+00:00` corresponds to the actual timestamp used for the Main Survey.

If secondary target files have been produced, then ledgers of secondary targets can additionally be generated using the following commands:

```
make_initial_mtl_ledger $CSCRATCH/dr9/1.1.1/targets/main/secondary/dark/targets-dark-secondary.fits --dest $CSCRATCH --obscon DARK --timestamp 2021-05-13T08:15:37+00:00 --numproc 8
make_initial_mtl_ledger $CSCRATCH/dr9/1.1.1/targets/main/secondary/bright/targets-bright-secondary.fits --dest $CSCRATCH --obscon BRIGHT --timestamp 2021-05-13T08:15:37+00:00 --numproc 8
```

Finally, a similar approach can be used to create ledgers of backup targets. As a reminder, this will require switching to the version of the `desitarget` codebase used to process backup targets (`2.2.0` rather than `1.1.1`). Once this switch has been made, the relevant command is:

```
make_initial_mtl_ledger $CSCRATCH/gaiadr2/2.2.0/targets/main/resolve/backup/ --dest $CSCRATCH --obscon BACKUP --numproc 8
```

The ledgers that result from running each of the five `make_initial_mtl_ledger` commands in this section will be generated in the `$CSCRATCH/main/dark`, `$CSCRATCH/main/bright`, `$CSCRATCH/main/secondary/dark`, `$CSCRATCH/main/secondary/bright` and `$CSCRATCH/main/backup` directories. This directory structure reflects the  data model for the MTL ledgers described in [Schlafly et al. (2022)][Schlaflyurl].

At NERSC, using the specified 8 processors per node (`--numproc 8`) each of the `make_initial_mtl_ledger` commands takes about 30-40 minutes to run on a single Cori production node.

# Other tips and tricks for working with desitarget #

## General

- All `desitarget` executable scripts listed in the previous section have a `-h` flag that will 
describe how to use that code, e.g.:

```
select_targets -h
```

## select_targets


- It is possible to use the `--tcnames` ("target class names") flag to select and set targeting bits for just one target class. For example, to select 
only the quasar target class:

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --nside 8 --healpixels 444 --numproc 4 --nosec --tcnames QSO
```

- Multiple target classeses can also be selected by passing a comma-separated list. For example, to 
select only the LRG and ELG target classes:

```
select_targets $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --nside 8 --healpixels 444 --numproc 4 --nosec --tcnames LRG,ELG
```

- Versions of `select_targets` also exist for commissioning (cmx) and Survey Validation (SV). An example of running the commissioning version of the code is:

```
select_cmx_targets $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --nside 8 --healpixels 444 --numproc 4
```

- An example of running the SV version of the code would be:

```
select_sv_targets $LS_DIR/south/sweep/9.0 $CSCRATCH -s2 $LS_DIR/north/sweep/9.0 --nside 8 --healpixels 444 --numproc 4 --nosec --iteration 2
```

The SV code has many of the same options as `select_targets` but uses the additional flag `--iteration` to denote *which* version of SV is being run (e.g., the first iteration of SV, called SV1, corresponds to `--iteration 1`).

## run_target_qa

- The `tcnames` flag (see the previous section) can also be used to produce _web pages_ for a subset of targets. For example:

```
run_target_qa $CSCRATCH/bright.fits $WEB_DIR/1.1.1/main/desitargetQA-dr9-bright-1.1.1 --numproc 2 -w $CSCRATCH/pixweight-1-bright.fits --tcnames LRG,ELG

```

- It isn't necessary to use a pixweight map if one hasn't been made. Without a pixweight map, `run_target_qa` will produce the same output but with *equal* weighting for all areas of the Legacy Surveys imaging footprint. To achieve this, the `-w` flag can just be omitted, e.g.:

```
run_target_qa $CSCRATCH/bright.fits $WEB_DIR/1.1.1/main/desitargetQA-dr9-bright-1.1.1
```


- Producing plots is the slowest part of making the targeting QA web pages. The pages can be produced without generating any new plots by adding the `--noplots` option, e.g.:

```
run_target_qa $CSCRATCH/bright.fits $WEB_DIR/1.1.1/main/desitargetQA-dr9-bright-1.1.1 --noplots
```

- If a pixweight map _is_ passed, `run_target_qa` produces plots of systematics across the imaging footprint by default. It can take some time to produce these plots, but they can be turned off with the `--nosystematics` flag, e.g.:

```
run_target_qa $CSCRATCH/bright.fits $WEB_DIR/1.1.1/main/desitargetQA-dr9-bright-1.1.1 --nosystematics
```

